# Introducing Bovine

The current library is the part of bovine that contains the necessities to build an ActivityPub Client and a bunch of utils that are necessary to work with ActivityPub, for example handling of various authentication/authorization methods or common methods to handle ActivityStreams object, e.g. determining if it is addressed to the Public.

In order to use Bovine, we first need to create a BovineActor object:

In [1]:
from bovine import BovineActor
actor = BovineActor("../h.toml")
await actor.init()

## The inbox

The next example illustrate how to iterate of the inbox of an Actor. It should be noted that the inbox is a CollectionHelper type of object. The max number can be set to None, however this will lead to stupid long run times, as there is a lot of stuff in the typical ActivityPub Inbox.

In [3]:
inbox = await actor.inbox()
await inbox.summary()


Items loaded 10

   0: Delete    : --- unknown  ---
   1: Delete    : https://tecnfy.com/users/andy/statuses/109661143600364804
   2: Create    : @Bundesregierung https://www.hartziv.org/news/20230310-buergergeld-49-euro-ticke
   3: Create    : @Volksverpetzer Wann kümmert ihr euch um eure Datenschutzprobleme? Da auch mal e
   4: Update    : While this appaears concerning I have no f&#39;ing idea what &quot;Google Cloud 
   5: Update    : How long has it been since you ate popcorn?
#EvanPoll #Poll
   6: Create    : While this appaears concerning I have no f&#39;ing idea what &quot;Google Cloud 
   7: Announce  : --- unknown  ---
   8: Delete    : --- unknown  ---
   9: Create    : @michcia @domi @Dee json-ld api :)))
no joke, what if the api returned a flatten


In [6]:
from bovine.activitystreams.utils.print import print_activity

async for x in inbox.iterate(max_number=3):
    print_activity(x)


Type: Create    , Id: https://mastodon.gamedevalliance.fr/users/nighten/statuses/110055117241861739/activity

Type: Note

Content
@evan Where is the &quot;never&quot; option evan?? &gt;:(

Type: Create    , Id: https://mastodon.social/users/airwhale/statuses/110055112253259416/activity

Type: Note

Content
@SwiftOnSecurity 
&quot;Without CatGirl, no CatWoman there can be.&quot; - Yoda

Type: Create    , Id: https://chaos.social/users/gnarly_parker/statuses/110055102348797640/activity

Type: Note

Content
@atomicpoet You mean every time? It always stayed a frog. 😢



## The Outbox

Let's write a note. We will write our note to one of my test accounts `@themilkman@mas.to`. For more options on what to do with notes, consult the `ObjectBuilder` class.

In [2]:
activity_factory, object_factory = actor.factories
note = object_factory.note("Hello").add_to("https://mas.to/users/themilkman").build()
create = activity_factory.create(note).build()

create

{'@context': ['https://www.w3.org/ns/activitystreams',
  {'inReplyToAtomUri': 'ostatus:inReplyToAtomUri',
   'conversation': 'ostatus:conversation',
   'ostatus': 'http://ostatus.org#'}],
 'id': None,
 'type': 'Create',
 'actor': 'https://mymath.rocks/endpoints/SYn3cl_N4HAPfPHgo2x37XunLEmhV9LnxCggcYwyec0',
 'object': {'@context': 'https://www.w3.org/ns/activitystreams',
  'attributedTo': 'https://mymath.rocks/endpoints/SYn3cl_N4HAPfPHgo2x37XunLEmhV9LnxCggcYwyec0',
  'type': 'Note',
  'inReplyTo': None,
  'content': 'Hello',
  'published': '2023-03-20T14:17:58Z',
  'to': ['https://mas.to/users/themilkman'],
  'cc': []},
 'published': '2023-03-20T14:17:58Z',
 'to': ['https://mas.to/users/themilkman'],
 'cc': []}

Let's now send our note!

In [23]:
await actor.send_to_outbox(create)

<ClientResponse(https://mymath.rocks/endpoints/fh07LGxmCxPlo_uHqfmWrzHHBRFjlyz7MyC6ge4eM4c) [201 ]>
<CIMultiDictProxy('Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Mon, 20 Mar 2023 10:05:36 GMT', 'Content-Type': 'application/json', 'Content-Length': '20', 'Connection': 'keep-alive', 'Location': 'https://mymath.rocks/objects/3853a3d6-9612-4155-88e0-e6c7cbeec882')>

We realize that this was not displayed as a direct message, so we fix it. For this we need to add the target actor to the mentions. Don't ask me why, ask Mastodon!

In [7]:
note = object_factory.note("Can you see this?").add_to("https://mas.to/users/themilkman").with_mention("https://mas.to/users/themilkman").build()
note["audience"] = ["as:Public"]
create = activity_factory.create(note).build()
create["audience"] = ["as:Public"]
create
await actor.send_to_outbox(create)

<ClientResponse(https://mymath.rocks/endpoints/fh07LGxmCxPlo_uHqfmWrzHHBRFjlyz7MyC6ge4eM4c) [201 ]>
<CIMultiDictProxy('Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Mon, 20 Mar 2023 14:18:55 GMT', 'Content-Type': 'application/json', 'Content-Length': '20', 'Connection': 'keep-alive', 'Location': 'https://mymath.rocks/objects/0d81f39e-3141-4a88-b2cd-746a06699374')>

In [8]:
create

{'@context': ['https://www.w3.org/ns/activitystreams',
  {'inReplyToAtomUri': 'ostatus:inReplyToAtomUri',
   'conversation': 'ostatus:conversation',
   'ostatus': 'http://ostatus.org#'}],
 'id': None,
 'type': 'Create',
 'actor': 'https://mymath.rocks/endpoints/SYn3cl_N4HAPfPHgo2x37XunLEmhV9LnxCggcYwyec0',
 'object': {'@context': 'https://www.w3.org/ns/activitystreams',
  'attributedTo': 'https://mymath.rocks/endpoints/SYn3cl_N4HAPfPHgo2x37XunLEmhV9LnxCggcYwyec0',
  'type': 'Note',
  'inReplyTo': None,
  'content': 'Can you see this?',
  'published': '2023-03-20T14:18:54Z',
  'to': ['https://mas.to/users/themilkman'],
  'cc': [],
  'tag': [{'href': 'https://mas.to/users/themilkman',
    'name': 'https://mas.to/users/themilkman',
    'type': 'Mention'}],
  'audience': ['as:Public']},
 'published': '2023-03-20T14:18:54Z',
 'to': ['https://mas.to/users/themilkman'],
 'cc': [],
 'audience': ['as:Public']}

In [9]:
response.headers

NameError: name 'response' is not defined

## Proxying elements

There are two ways to get objects with a BovineActor. First, there is `actor.get`, which performs a HTML request to the resource. Depending on how your BovineActor does authentication, you might not be able to resolve the object. To work around this, you can use `actor.proxy_element`, which lets the ActivityPub Server perform the request and then proxies the result to you. Also the cache of the ActivityPub server is used here.

In [9]:
await actor.get('https://mastodon.social/actor')

{'@context': ['https://www.w3.org/ns/activitystreams',
  'https://w3id.org/security/v1',
  {'manuallyApprovesFollowers': 'as:manuallyApprovesFollowers',
   'toot': 'http://joinmastodon.org/ns#',
   'featured': {'@id': 'toot:featured', '@type': '@id'},
   'featuredTags': {'@id': 'toot:featuredTags', '@type': '@id'},
   'alsoKnownAs': {'@id': 'as:alsoKnownAs', '@type': '@id'},
   'movedTo': {'@id': 'as:movedTo', '@type': '@id'},
   'schema': 'http://schema.org#',
   'PropertyValue': 'schema:PropertyValue',
   'value': 'schema:value',
   'discoverable': 'toot:discoverable',
   'Device': 'toot:Device',
   'Ed25519Signature': 'toot:Ed25519Signature',
   'Ed25519Key': 'toot:Ed25519Key',
   'Curve25519Key': 'toot:Curve25519Key',
   'EncryptedMessage': 'toot:EncryptedMessage',
   'publicKeyBase64': 'toot:publicKeyBase64',
   'deviceId': 'toot:deviceId',
   'claim': {'@type': '@id', '@id': 'toot:claim'},
   'fingerprintKey': {'@type': '@id', '@id': 'toot:fingerprintKey'},
   'identityKey': {'@t

In [8]:
await actor.proxy_element('https://mastodon.social/actor')

{'@context': ['https://www.w3.org/ns/activitystreams',
  'https://w3id.org/security/v1',
  {'Curve25519Key': 'toot:Curve25519Key',
   'Device': 'toot:Device',
   'Ed25519Key': 'toot:Ed25519Key',
   'Ed25519Signature': 'toot:Ed25519Signature',
   'EncryptedMessage': 'toot:EncryptedMessage',
   'PropertyValue': 'schema:PropertyValue',
   'alsoKnownAs': {'@id': 'as:alsoKnownAs', '@type': '@id'},
   'cipherText': 'toot:cipherText',
   'claim': {'@id': 'toot:claim', '@type': '@id'},
   'deviceId': 'toot:deviceId',
   'devices': {'@id': 'toot:devices', '@type': '@id'},
   'discoverable': 'toot:discoverable',
   'featured': {'@id': 'toot:featured', '@type': '@id'},
   'featuredTags': {'@id': 'toot:featuredTags', '@type': '@id'},
   'fingerprintKey': {'@id': 'toot:fingerprintKey', '@type': '@id'},
   'identityKey': {'@id': 'toot:identityKey', '@type': '@id'},
   'manuallyApprovesFollowers': 'as:manuallyApprovesFollowers',
   'messageFranking': 'toot:messageFranking',
   'messageType': 'toot:me